In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})
sys.path.append('../src/')
from Biologging_Toolkit.applications.Wind import Wind
from Biologging_Toolkit.plot.wind import *

#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [ ]:
depids = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
path = 'D:/individus_brut/individus'

In [ ]:
freqs = [50, 100, 200, 300, 500, 1000, 2500, 5000, 8000, 10000, 12000, 15000, 16000]

### PLOT DISTRIBUTIONS

In [ ]:
plot_contour_upwards_downwards(depids)

#### Get estimations from different models

In [ ]:
plot_accuracy_frequencies(depids, freqs, path = 'frequency_analysis')

### Fine scale training

In [ ]:
def func1(x, a, b, c) :
    return a*(x-c) + b
def func2(x, a, b, c, d) :
    return a*(x-d)**2+b*(x-d)+c
bounds = [[[-np.inf, -np.inf, -1e-18], [np.inf, np.inf, 1e-18]], [[-np.inf, -np.inf, -np.inf, -1e-18], [np.inf, np.inf, np.inf, 1e-18]]]

In [ ]:
for depid in depids :
    inst = Wind(depid, path = os.path.join(path, depid),
                acoustic_path = os.path.join(path, depid, 'dives'))
    depth = [50,100,150,200,250,300,350,400,450,500]
    df = pd.DataFrame()
    for min_depth, max_depth in zip(depth[:-1], depth[1:]):
        inst.get_fine_scale(depth = [min_depth, max_depth])
        inst.two_regime(func1, func2, bounds)
        df = pd.concat((df, pd.DataFrame(inst.wind_model_stats, index = [min_depth])))
    df.to_csv(f'C:/Users/grosmaan/Desktop/{depid}_50_500.csv')
    depth = [0,5,10,15,20,25,30,35,40,45,50,55]
    df = pd.DataFrame()
    for min_depth, max_depth in zip(depth[:-1], depth[1:]):
        inst.get_fine_scale(depth = [min_depth, max_depth])
        inst.two_regime(func1, func2, bounds)
        df = pd.concat((df, pd.DataFrame(inst.wind_model_stats, index = [min_depth])))
    df.to_csv(f'C:/Users/grosmaan/Desktop/{depid}_5_50.csv')

In [ ]:
inst.test_split

In [ ]:
inst.wind_model_stats

In [ ]:
df.to_csv(f'C:/Users/grosmaan/Desktop/{depids[5]}_50_500.csv')

### Get model performance

In [ ]:
## Save empirical model
df = pd.DataFrame()
freq = 5000
for depid in depids:
        inst = Wind(depids, path = [os.path.join(path, depid) for depid in depids], 
                    data = f'dive_mean_{int(freq)}', 
                    test_depid = depid, method = 'Hildebrand')
        inst.depid_fit()
        df[depid] = inst.wind_model_stats
df.to_csv(f'C:/Users/grosm/Desktop/thèse/Results/logarithmic_dive_{freq}.csv', index = None)

In [ ]:
inst.wind_model_stats

In [ ]:
log_up = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/logarithmic_upwards_5000.csv')
log_down = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/logarithmic_downwards_5000.csv')
quad_up = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/quadratic_upwards_5000.csv')
quad_down = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/quadratic_downwards_5000.csv')

In [ ]:
metric = pd.DataFrame({'up quadratic' : quad_up.mean(axis = 1), 
                       'down quadratic' : quad_down.mean(axis = 1),
                       'up logarithmic' : log_up.mean(axis = 1), 
                       'down logarithmic' : log_down.mean(axis = 1)})

In [ ]:
metric = metric.T
metric.columns = ['mae', 'rmse', 'r1', 'var', 'std']
metric = metric.round(2)
metric = metric.map(lambda x: f"{x:.2f}")
metric.to_latex('C:/Users/grosm/Desktop/thèse/Results/empirical_metric.tex', columns = ['mae', 'rmse', 'std'])

In [ ]:
metric